<a href="https://colab.research.google.com/github/mcabrera2004/DeepSeekTranslatorAI/blob/main/TranslatorAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Si no está instalado CUDA 11.8
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit

--2025-03-08 04:59:48--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.210.244.213, 23.210.244.203
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.210.244.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run’

cuda_11.8.0_520.61. 100%[===================>]   4.04G   126MB/s    in 39s     

2025-03-08 05:00:28 (105 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run’ saved [4336730777/4336730777]



In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!pip install pillow pdf2image python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install torch==2.0.1+cu118 torchaudio==2.0.2+cu118 torchvision==0.15.2+cu118 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install git+https://github.com/deepseek-ai/DeepSeek-VL2.git

  Cloning https://github.com/deepseek-ai/DeepSeek-VL2.git to /tmp/pip-req-build-jgx25bm6
  Running command git clone --filter=blob:none --quiet https://github.com/deepseek-ai/DeepSeek-VL2.git /tmp/pip-req-build-jgx25bm6
  Resolved https://github.com/deepseek-ai/DeepSeek-VL2.git to commit ef9f91e2b6426536b83294c11742c27be66361b1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.i

In [ ]:
from google.colab import files

# Subir el archivo PDF
uploaded = files.upload()

# Obtener el nombre del archivo subido
ruta_pdf = list(uploaded.keys())[0]
print(f"Archivo subido: {ruta_pdf}")

Saving Weir_Egg_removed.pdf to Weir_Egg_removed.pdf
Archivo subido: Weir_Egg_removed.pdf


In [ ]:
import torch

def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Usando GPU para inferencia:", torch.cuda.get_device_name(0))
    else:
        device = torch.device("cpu")
        print("GPU no disponible, usando CPU para inferencia")
    return device

device = get_device()

GPU no disponible, usando CPU para inferencia


In [ ]:
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM

model_path = "deepseek-ai/deepseek-vl2-tiny"
vl_chat_processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

# Cargar modelo - para GPU usamos half precision
print(f"Cargando modelo en {device}...")
torch_dtype = torch.float16 if device.type == "cuda" else torch.float32
vl_gpt = DeepseekVLV2ForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch_dtype
)

# Mover el modelo al dispositivo
vl_gpt = vl_gpt.to(device)
vl_gpt.eval()  # Modo de inferencia
print("Modelo cargado")

ModuleNotFoundError: No module named 'deepseek_vl2'

In [ ]:
from pdf2image import convert_from_path
import os

def pdf_a_imagenes(ruta_pdf, ruta_salida="imagenes_paginas"):
    if not os.path.exists(ruta_salida):
        os.makedirs(ruta_salida)

    paginas = convert_from_path(ruta_pdf, dpi=300)
    rutas_imagenes = []

    for i, pagina in enumerate(paginas):
        ruta_imagen = os.path.join(ruta_salida, f"pagina_{i+1}.jpg")
        pagina.save(ruta_imagen, "JPEG")
        rutas_imagenes.append(ruta_imagen)

    return rutas_imagenes

In [ ]:
from PIL import Image

def extraer_texto_con_deepseek(ruta_imagen):
    image = Image.open(ruta_imagen).convert("RGB")

    conversation = [
        {
            "role": "<|User|>",
            "content": "<image>\n|ref|>Extrae el texto de la imagen.<|/ref|>",
            "images": [image],
        },
        {"role": "<|Assistant|>", "content": ""},
    ]

    model_inputs = vl_chat_processor(
        conversations=conversation,
        images=[image],
        force_batchify=True,
        system_prompt=""
    ).to(device)  # Mover inputs al dispositivo configurado

    inputs_embeds = vl_gpt.prepare_inputs_embeds(**model_inputs)

    with torch.no_grad():
        outputs = vl_gpt.language.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=model_inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=2048,
            do_sample=False,
            use_cache=True
        )

    response = tokenizer.decode(outputs[0].tolist(), skip_special_tokens=True)
    return response

In [ ]:
def traducir_texto(texto):
    conversation = [
        {"role": "<|User|>", "content": f"Traduce el siguiente texto al español: {texto}"},
        {"role": "<|Assistant|>", "content": ""},
    ]

    model_inputs = vl_chat_processor(
        conversations=conversation,
        force_batchify=True,
        system_prompt=""
    ).to(device)  # Mover inputs al dispositivo configurado

    inputs_embeds = vl_gpt.prepare_inputs_embeds(**model_inputs)

    with torch.no_grad():
        outputs = vl_gpt.language.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=model_inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=2048,
            do_sample=False,
            use_cache=True
        )

    response = tokenizer.decode(outputs[0].tolist(), skip_special_tokens=True)
    return response

In [ ]:
from docx import Document

def traducir_y_guardar_en_word(ruta_pdf, nombre_archivo="libro_traducido.docx"):
    imagenes = pdf_a_imagenes(ruta_pdf)
    doc = Document()
    doc.add_heading("Libro Traducido", level=1)

    buffer = ""

    for i, imagen in enumerate(imagenes):
        print(f"🔍 Procesando página {i+1}...")
        texto_extraido = extraer_texto_con_deepseek(imagen)

        if buffer:
            texto_extraido = buffer + " " + texto_extraido

        if not texto_extraido.strip().endswith("."):
            partes = texto_extraido.rsplit(". ", 1)
            if len(partes) > 1:
                texto_extraido, buffer = partes[0] + ".", partes[1]
            else:
                buffer = texto_extraido
                texto_extraido = ""
        else:
            buffer = ""

        if texto_extraido.strip():
            traduccion = traducir_texto(texto_extraido)
            doc.add_paragraph(traduccion)

        doc.save(nombre_archivo)

    if buffer.strip():
        traduccion = traducir_texto(buffer)
        doc.add_paragraph(traduccion)
        doc.save(nombre_archivo)

    print(f"✅ Traducción completada. Guardado en {nombre_archivo}")

In [ ]:
if __name__ == "__main__":
    traducir_y_guardar_en_word(ruta_pdf)

In [ ]:
from google.colab import files

# Descargar el archivo DOCX generado
files.download("libro_traducido.docx")